# Phân tích và Trực quan hóa Dữ liệu Khách sạn

Notebook này sẽ phân tích và trực quan hóa dữ liệu từ file `merged_add_tripadvisior_trip.csv`


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Thiết lập style cho biểu đồ
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Hiển thị tiếng Việt trong biểu đồ
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("Đã import các thư viện thành công!")


In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv('merged_add_tripadvisior_trip.csv')

# Hiển thị thông tin cơ bản về dataset
print("Kích thước dataset:", df.shape)
print("\nCác cột trong dataset:")
print(df.columns.tolist())
print("\nThông tin cơ bản:")
print(df.info())
print("\n5 dòng đầu tiên:")
df.head()


In [ ]:
# Thống kê mô tả cho các cột số
print("Thống kê mô tả cho các cột số:")
df.describe()


## 1. Phân tích Phân phối Giá (Final Price)


In [ ]:
# Làm sạch dữ liệu giá - loại bỏ các giá trị null và giá trị bất thường
price_data = df['Final Price'].dropna()
price_data = price_data[price_data > 0]  # Loại bỏ giá <= 0

# Tạo figure với nhiều subplot
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Histogram phân phối giá
axes[0, 0].hist(price_data, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Phân phối Giá (Final Price)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Giá (VND)')
axes[0, 0].set_ylabel('Tần suất')
axes[0, 0].grid(True, alpha=0.3)

# 2. Box plot để xem outliers
axes[0, 1].boxplot(price_data, vert=True)
axes[0, 1].set_title('Box Plot - Phân phối Giá', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Giá (VND)')
axes[0, 1].grid(True, alpha=0.3)

# 3. Histogram với log scale (để xem rõ hơn)
axes[1, 0].hist(np.log10(price_data), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_title('Phân phối Giá (Log Scale)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Log10(Giá)')
axes[1, 0].set_ylabel('Tần suất')
axes[1, 0].grid(True, alpha=0.3)

# 4. Thống kê giá theo percentile
percentiles = [10, 25, 50, 75, 90, 95, 99]
percentile_values = [np.percentile(price_data, p) for p in percentiles]
axes[1, 1].bar(range(len(percentiles)), percentile_values, color='coral', alpha=0.7)
axes[1, 1].set_title('Giá theo Percentile', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Percentile')
axes[1, 1].set_ylabel('Giá (VND)')
axes[1, 1].set_xticks(range(len(percentiles)))
axes[1, 1].set_xticklabels([f'{p}%' for p in percentiles])
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Giá trung bình: {price_data.mean():,.0f} VND")
print(f"Giá trung vị: {price_data.median():,.0f} VND")
print(f"Giá tối thiểu: {price_data.min():,.0f} VND")
print(f"Giá tối đa: {price_data.max():,.0f} VND")


## 2. Phân tích theo Địa điểm (Search Location)


In [ ]:
# Phân tích theo địa điểm
location_counts = df['Search Location'].value_counts()
location_price = df.groupby('Search Location')['Final Price'].agg(['mean', 'median', 'count']).sort_values('count', ascending=False)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Số lượng khách sạn theo địa điểm
axes[0, 0].barh(location_counts.index, location_counts.values, color='steelblue', alpha=0.7)
axes[0, 0].set_title('Số lượng Khách sạn theo Địa điểm', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Số lượng')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. Giá trung bình theo địa điểm
axes[0, 1].barh(location_price.index, location_price['mean'], color='coral', alpha=0.7)
axes[0, 1].set_title('Giá Trung bình theo Địa điểm', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Giá Trung bình (VND)')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Giá trung vị theo địa điểm
axes[1, 0].barh(location_price.index, location_price['median'], color='lightgreen', alpha=0.7)
axes[1, 0].set_title('Giá Trung vị theo Địa điểm', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Giá Trung vị (VND)')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. So sánh giá trung bình và trung vị
x = np.arange(len(location_price.index))
width = 0.35
axes[1, 1].bar(x - width/2, location_price['mean'], width, label='Trung bình', alpha=0.7, color='coral')
axes[1, 1].bar(x + width/2, location_price['median'], width, label='Trung vị', alpha=0.7, color='lightgreen')
axes[1, 1].set_title('So sánh Giá Trung bình và Trung vị', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Địa điểm')
axes[1, 1].set_ylabel('Giá (VND)')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(location_price.index, rotation=45, ha='right')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nThống kê theo địa điểm:")
print(location_price)


## 3. Phân tích Rating và Review


In [ ]:
# Phân tích rating
# Chuyển đổi Rating_Clean sang số, loại bỏ các giá trị không hợp lệ
df['Rating_Clean'] = pd.to_numeric(df['Rating_Clean'], errors='coerce')
rating_data = df['Rating_Clean'].dropna()
rating_data = rating_data[(rating_data >= 0) & (rating_data <= 10)]  # Chỉ lấy rating từ 0-10

# Chuyển đổi Review Count sang số
df['Review Count'] = pd.to_numeric(df['Review Count'], errors='coerce')
review_count_data = df['Review Count'].dropna()
review_count_data = review_count_data[review_count_data >= 0]  # Chỉ lấy giá trị >= 0

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Phân phối Rating
if len(rating_data) > 0:
    axes[0, 0].hist(rating_data, bins=20, edgecolor='black', alpha=0.7, color='purple')
    axes[0, 0].set_title('Phân phối Rating', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Rating')
    axes[0, 0].set_ylabel('Tần suất')
    axes[0, 0].axvline(rating_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Trung bình: {rating_data.mean():.2f}')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
else:
    axes[0, 0].text(0.5, 0.5, 'Không có dữ liệu Rating', ha='center', va='center', transform=axes[0, 0].transAxes)
    axes[0, 0].set_title('Phân phối Rating', fontsize=14, fontweight='bold')

# 2. Phân phối Review Count
if len(review_count_data) > 0:
    axes[0, 1].hist(review_count_data, bins=50, edgecolor='black', alpha=0.7, color='orange')
    axes[0, 1].set_title('Phân phối Số lượng Review', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Số lượng Review')
    axes[0, 1].set_ylabel('Tần suất')
    axes[0, 1].grid(True, alpha=0.3)
else:
    axes[0, 1].text(0.5, 0.5, 'Không có dữ liệu Review', ha='center', va='center', transform=axes[0, 1].transAxes)
    axes[0, 1].set_title('Phân phối Số lượng Review', fontsize=14, fontweight='bold')

# 3. Rating vs Review Count (scatter plot)
valid_data = df[['Rating_Clean', 'Review Count']].dropna()
valid_data = valid_data[(valid_data['Rating_Clean'] >= 0) & (valid_data['Rating_Clean'] <= 10)]
valid_data = valid_data[valid_data['Review Count'] >= 0]

if len(valid_data) > 0:
    axes[1, 0].scatter(valid_data['Review Count'], valid_data['Rating_Clean'], alpha=0.5, s=20)
    axes[1, 0].set_title('Mối quan hệ giữa Rating và Số lượng Review', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Số lượng Review')
    axes[1, 0].set_ylabel('Rating')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'Không có dữ liệu hợp lệ', ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].set_title('Mối quan hệ giữa Rating và Số lượng Review', fontsize=14, fontweight='bold')

# 4. Box plot Rating theo địa điểm
location_rating = df.groupby('Search Location')['Rating_Clean'].apply(lambda x: pd.to_numeric(x, errors='coerce').dropna().tolist())
location_rating = {loc: ratings for loc, ratings in location_rating.items() if len(ratings) > 0 and all(0 <= r <= 10 for r in ratings)}

if len(location_rating) > 0:
    box_data = [location_rating[loc] for loc in location_rating.keys()]
    axes[1, 1].boxplot(box_data, labels=list(location_rating.keys()))
    axes[1, 1].set_title('Phân phối Rating theo Địa điểm', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Địa điểm')
    axes[1, 1].set_ylabel('Rating')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
else:
    axes[1, 1].text(0.5, 0.5, 'Không có dữ liệu Rating theo địa điểm', ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title('Phân phối Rating theo Địa điểm', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

if len(rating_data) > 0:
    print(f"Rating trung bình: {rating_data.mean():.2f}")
    print(f"Rating trung vị: {rating_data.median():.2f}")
else:
    print("Không có dữ liệu Rating hợp lệ")

if len(review_count_data) > 0:
    print(f"Số lượng review trung bình: {review_count_data.mean():.0f}")
    print(f"Số lượng review trung vị: {review_count_data.median():.0f}")
else:
    print("Không có dữ liệu Review hợp lệ")


## 4. Phân tích Loại Phòng và Giường


In [ ]:
# Phân tích loại phòng
room_type_counts = df['Room Type'].value_counts().head(10)
bed_type_counts = df['Bed Type'].value_counts().head(10)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Top 10 loại phòng
axes[0, 0].barh(room_type_counts.index, room_type_counts.values, color='teal', alpha=0.7)
axes[0, 0].set_title('Top 10 Loại Phòng Phổ biến', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Số lượng')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. Top 10 loại giường
axes[0, 1].barh(bed_type_counts.index, bed_type_counts.values, color='salmon', alpha=0.7)
axes[0, 1].set_title('Top 10 Loại Giường Phổ biến', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Số lượng')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Giá trung bình theo loại phòng (top 10)
room_price = df.groupby('Room Type')['Final Price'].mean().sort_values(ascending=False).head(10)
axes[1, 0].barh(room_price.index, room_price.values, color='gold', alpha=0.7)
axes[1, 0].set_title('Giá Trung bình theo Loại Phòng (Top 10)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Giá Trung bình (VND)')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Phân phối số phòng
rooms_data = df['Rooms'].dropna()
axes[1, 1].hist(rooms_data, bins=range(1, int(rooms_data.max())+2), edgecolor='black', alpha=0.7, color='mediumpurple')
axes[1, 1].set_title('Phân phối Số lượng Phòng', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Số lượng Phòng')
axes[1, 1].set_ylabel('Tần suất')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Phân tích theo Quận/Huyện (District)


In [ ]:
# Phân tích theo quận/huyện
district_counts = df['District'].value_counts().head(15)
district_price = df.groupby('District')['Final Price'].agg(['mean', 'count']).sort_values('count', ascending=False).head(15)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Số lượng khách sạn theo quận
axes[0].barh(district_counts.index, district_counts.values, color='steelblue', alpha=0.7)
axes[0].set_title('Số lượng Khách sạn theo Quận/Huyện (Top 15)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Số lượng')
axes[0].grid(True, alpha=0.3, axis='x')

# 2. Giá trung bình theo quận
axes[1].barh(district_price.index, district_price['mean'], color='coral', alpha=0.7)
axes[1].set_title('Giá Trung bình theo Quận/Huyện (Top 15)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Giá Trung bình (VND)')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nTop 15 quận/huyện có nhiều khách sạn nhất:")
print(district_price)


## 6. Mối quan hệ giữa Giá và Rating


In [ ]:
# Phân tích mối quan hệ giữa giá và rating
# Đảm bảo Rating_Clean đã được chuyển đổi sang số (nếu chưa thì chuyển đổi)
if df['Rating_Clean'].dtype == 'object':
    df['Rating_Clean'] = pd.to_numeric(df['Rating_Clean'], errors='coerce')

price_rating_data = df[['Final Price', 'Rating_Clean']].dropna()
price_rating_data = price_rating_data[price_rating_data['Final Price'] > 0]
# Chỉ lấy rating trong khoảng hợp lệ (0-10)
price_rating_data = price_rating_data[(price_rating_data['Rating_Clean'] >= 0) & (price_rating_data['Rating_Clean'] <= 10)]

if len(price_rating_data) == 0:
    print("Không có dữ liệu hợp lệ để phân tích mối quan hệ giữa Giá và Rating")
else:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Scatter plot giá vs rating
    axes[0, 0].scatter(price_rating_data['Final Price'], price_rating_data['Rating_Clean'], 
                       alpha=0.3, s=10, color='blue')
    axes[0, 0].set_title('Mối quan hệ giữa Giá và Rating', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Giá (VND)')
    axes[0, 0].set_ylabel('Rating')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Scatter plot với log scale cho giá
    axes[0, 1].scatter(np.log10(price_rating_data['Final Price']), 
                       price_rating_data['Rating_Clean'], 
                       alpha=0.3, s=10, color='green')
    axes[0, 1].set_title('Mối quan hệ giữa Log(Giá) và Rating', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Log10(Giá)')
    axes[0, 1].set_ylabel('Rating')
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Giá trung bình theo rating (grouped)
    rating_bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    price_rating_data['Rating_Bin'] = pd.cut(price_rating_data['Rating_Clean'], bins=rating_bins)
    price_by_rating = price_rating_data.groupby('Rating_Bin')['Final Price'].mean()
    
    if len(price_by_rating) > 0:
        axes[1, 0].bar(range(len(price_by_rating)), price_by_rating.values, 
                       color='purple', alpha=0.7, edgecolor='black')
        axes[1, 0].set_title('Giá Trung bình theo Khoảng Rating', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Khoảng Rating')
        axes[1, 0].set_ylabel('Giá Trung bình (VND)')
        axes[1, 0].set_xticks(range(len(price_by_rating)))
        axes[1, 0].set_xticklabels([f'{int(interval.left)}-{int(interval.right)}' 
                                    for interval in price_by_rating.index], rotation=45, ha='right')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
    else:
        axes[1, 0].text(0.5, 0.5, 'Không có dữ liệu', ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Giá Trung bình theo Khoảng Rating', fontsize=14, fontweight='bold')

    # 4. Heatmap correlation
    correlation = price_rating_data[['Final Price', 'Rating_Clean']].corr()
    sns.heatmap(correlation, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
                ax=axes[1, 1], square=True, cbar_kws={"shrink": 0.8})
    axes[1, 1].set_title('Ma trận Tương quan', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Tính correlation
    corr_coef = price_rating_data['Final Price'].corr(price_rating_data['Rating_Clean'])
    print(f"\nHệ số tương quan giữa Giá và Rating: {corr_coef:.3f}")


## 7. Phân tích Discount và Original Price


In [ ]:
# Phân tích discount
discount_data = df[['Discount %', 'Original Price', 'Final Price']].dropna()
discount_data = discount_data[discount_data['Discount %'] > 0]  # Chỉ lấy các khách sạn có discount

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Phân phối discount
axes[0, 0].hist(discount_data['Discount %'], bins=30, edgecolor='black', alpha=0.7, color='red')
axes[0, 0].set_title('Phân phối Discount %', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Discount %')
axes[0, 0].set_ylabel('Tần suất')
axes[0, 0].axvline(discount_data['Discount %'].mean(), color='blue', linestyle='--', 
                   linewidth=2, label=f'Trung bình: {discount_data["Discount %"].mean():.2f}%')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. So sánh Original Price vs Final Price
sample_data = discount_data.sample(min(1000, len(discount_data)))  # Lấy mẫu để vẽ nhanh hơn
axes[0, 1].scatter(sample_data['Original Price'], sample_data['Final Price'], 
                   alpha=0.5, s=10, color='green')
axes[0, 1].plot([sample_data['Original Price'].min(), sample_data['Original Price'].max()],
                [sample_data['Original Price'].min(), sample_data['Original Price'].max()],
                'r--', linewidth=2, label='Đường y=x (không discount)')
axes[0, 1].set_title('So sánh Original Price vs Final Price', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Original Price (VND)')
axes[0, 1].set_ylabel('Final Price (VND)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Discount % theo địa điểm
location_discount = df.groupby('Search Location')['Discount %'].mean().sort_values(ascending=False)
axes[1, 0].barh(location_discount.index, location_discount.values, color='orange', alpha=0.7)
axes[1, 0].set_title('Discount % Trung bình theo Địa điểm', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Discount % Trung bình')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Số lượng khách sạn có discount
has_discount = (df['Discount %'] > 0).sum()
no_discount = (df['Discount %'] == 0).sum()
axes[1, 1].pie([has_discount, no_discount], labels=['Có Discount', 'Không có Discount'],
               autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'lightblue'])
axes[1, 1].set_title('Tỷ lệ Khách sạn có Discount', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nSố lượng khách sạn có discount: {has_discount}")
print(f"Số lượng khách sạn không có discount: {no_discount}")
print(f"Discount trung bình: {discount_data['Discount %'].mean():.2f}%")
print(f"Discount tối đa: {discount_data['Discount %'].max():.2f}%")


## 8. Phân tích Số lượng Khách và Phòng


In [ ]:
# Phân tích số lượng khách và phòng
guest_data = df[['Adults', 'Children', 'Total_Guests', 'Rooms', 'Final Price']].dropna()
guest_data = guest_data[guest_data['Final Price'] > 0]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Phân phối số người lớn
axes[0, 0].hist(guest_data['Adults'], bins=range(1, int(guest_data['Adults'].max())+2), 
                edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_title('Phân phối Số lượng Người lớn', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Số lượng Người lớn')
axes[0, 0].set_ylabel('Tần suất')
axes[0, 0].grid(True, alpha=0.3)

# 2. Phân phối số trẻ em
axes[0, 1].hist(guest_data['Children'], bins=range(0, int(guest_data['Children'].max())+2), 
                edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_title('Phân phối Số lượng Trẻ em', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Số lượng Trẻ em')
axes[0, 1].set_ylabel('Tần suất')
axes[0, 1].grid(True, alpha=0.3)

# 3. Giá trung bình theo số lượng khách
price_by_guests = guest_data.groupby('Total_Guests')['Final Price'].mean()
axes[1, 0].plot(price_by_guests.index, price_by_guests.values, marker='o', 
                linewidth=2, markersize=8, color='green')
axes[1, 0].set_title('Giá Trung bình theo Tổng số Khách', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Tổng số Khách')
axes[1, 0].set_ylabel('Giá Trung bình (VND)')
axes[1, 0].grid(True, alpha=0.3)

# 4. Giá trung bình theo số phòng
price_by_rooms = guest_data.groupby('Rooms')['Final Price'].mean()
axes[1, 1].bar(price_by_rooms.index, price_by_rooms.values, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Giá Trung bình theo Số lượng Phòng', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Số lượng Phòng')
axes[1, 1].set_ylabel('Giá Trung bình (VND)')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nThống kê về khách:")
print(f"Số người lớn trung bình: {guest_data['Adults'].mean():.1f}")
print(f"Số trẻ em trung bình: {guest_data['Children'].mean():.1f}")
print(f"Tổng số khách trung bình: {guest_data['Total_Guests'].mean():.1f}")
print(f"Số phòng trung bình: {guest_data['Rooms'].mean():.1f}")


## 9. Phân tích Diện tích và Tiện nghi


In [ ]:
# Phân tích diện tích
# Đảm bảo Rating_Clean đã được chuyển đổi sang số (nếu chưa thì chuyển đổi)
if df['Rating_Clean'].dtype == 'object':
    df['Rating_Clean'] = pd.to_numeric(df['Rating_Clean'], errors='coerce')

area_data = df[['Area_m2_cleaned', 'Final Price', 'Rating_Clean']].dropna()
area_data = area_data[area_data['Area_m2_cleaned'] > 0]
area_data = area_data[area_data['Final Price'] > 0]
# Chỉ lấy rating trong khoảng hợp lệ (0-10)
area_data = area_data[(area_data['Rating_Clean'] >= 0) & (area_data['Rating_Clean'] <= 10)]

if len(area_data) == 0:
    print("Không có dữ liệu hợp lệ để phân tích diện tích")
else:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Phân phối diện tích
    axes[0, 0].hist(area_data['Area_m2_cleaned'], bins=50, edgecolor='black', alpha=0.7, color='teal')
    axes[0, 0].set_title('Phân phối Diện tích (m²)', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Diện tích (m²)')
    axes[0, 0].set_ylabel('Tần suất')
    axes[0, 0].axvline(area_data['Area_m2_cleaned'].mean(), color='red', linestyle='--', 
                       linewidth=2, label=f'Trung bình: {area_data["Area_m2_cleaned"].mean():.1f} m²')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Mối quan hệ giữa diện tích và giá
    sample_area = area_data.sample(min(1000, len(area_data)))
    axes[0, 1].scatter(sample_area['Area_m2_cleaned'], sample_area['Final Price'], 
                       alpha=0.5, s=10, color='blue')
    axes[0, 1].set_title('Mối quan hệ giữa Diện tích và Giá', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Diện tích (m²)')
    axes[0, 1].set_ylabel('Giá (VND)')
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Giá trên m²
    area_data['Price_per_m2'] = area_data['Final Price'] / area_data['Area_m2_cleaned']
    axes[1, 0].hist(area_data['Price_per_m2'], bins=50, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 0].set_title('Phân phối Giá trên m²', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Giá trên m² (VND/m²)')
    axes[1, 0].set_ylabel('Tần suất')
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Mối quan hệ giữa diện tích và rating
    axes[1, 1].scatter(sample_area['Area_m2_cleaned'], sample_area['Rating_Clean'], 
                       alpha=0.5, s=10, color='purple')
    axes[1, 1].set_title('Mối quan hệ giữa Diện tích và Rating', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Diện tích (m²)')
    axes[1, 1].set_ylabel('Rating')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nDiện tích trung bình: {area_data['Area_m2_cleaned'].mean():.1f} m²")
    print(f"Diện tích trung vị: {area_data['Area_m2_cleaned'].median():.1f} m²")
    print(f"Giá trên m² trung bình: {area_data['Price_per_m2'].mean():,.0f} VND/m²")


## 10. Tổng kết và Insights


In [ ]:
# Tạo bảng tổng kết
# Đảm bảo các cột số đã được chuyển đổi
if df['Rating_Clean'].dtype == 'object':
    df['Rating_Clean'] = pd.to_numeric(df['Rating_Clean'], errors='coerce')
if df['Review Count'].dtype == 'object':
    df['Review Count'] = pd.to_numeric(df['Review Count'], errors='coerce')

# Tính toán các giá trị với xử lý NaN
rating_mean = df['Rating_Clean'].mean()
rating_mean_str = f"{rating_mean:.2f}" if pd.notna(rating_mean) else "N/A"

review_mean = df['Review Count'].mean()
review_mean_str = f"{review_mean:.0f}" if pd.notna(review_mean) else "N/A"

summary_data = {
    'Metric': [
        'Tổng số bản ghi',
        'Số lượng địa điểm',
        'Số lượng quận/huyện',
        'Giá trung bình (VND)',
        'Giá trung vị (VND)',
        'Rating trung bình',
        'Số review trung bình',
        'Diện tích trung bình (m²)',
        'Số phòng trung bình',
        'Số khách trung bình'
    ],
    'Value': [
        len(df),
        df['Search Location'].nunique(),
        df['District'].nunique(),
        f"{df['Final Price'].mean():,.0f}",
        f"{df['Final Price'].median():,.0f}",
        rating_mean_str,
        review_mean_str,
        f"{df['Area_m2_cleaned'].mean():.1f}",
        f"{df['Rooms'].mean():.1f}",
        f"{df['Total_Guests'].mean():.1f}"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("=" * 50)
print("TỔNG KẾT DỮ LIỆU")
print("=" * 50)
print(summary_df.to_string(index=False))
print("\n" + "=" * 50)

# Top insights
print("\nTOP INSIGHTS:")
print("=" * 50)
print(f"1. Địa điểm phổ biến nhất: {df['Search Location'].value_counts().index[0]}")
print(f"2. Quận/huyện phổ biến nhất: {df['District'].value_counts().index[0]}")
print(f"3. Loại phòng phổ biến nhất: {df['Room Type'].value_counts().index[0]}")
print(f"4. Khách sạn đắt nhất: {df.loc[df['Final Price'].idxmax(), 'Hotel Name']} - {df['Final Price'].max():,.0f} VND")
print(f"5. Khách sạn rẻ nhất: {df.loc[df['Final Price'].idxmin(), 'Hotel Name']} - {df['Final Price'].min():,.0f} VND")

# Kiểm tra rating trước khi tìm max
rating_valid = df['Rating_Clean'].dropna()
if len(rating_valid) > 0:
    max_rating_idx = rating_valid.idxmax()
    print(f"6. Khách sạn có rating cao nhất: {df.loc[max_rating_idx, 'Hotel Name']} - {rating_valid.max():.1f}")
else:
    print("6. Không có dữ liệu rating hợp lệ")
